In [1]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\MLPROJ_test\\reasearch'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\MLPROJ_test'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_csv: Path
    test_csv: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str


In [7]:
from src.mlproject1.constants import *
from src.mlproject1.utils.common import read_yaml,create_directories
import os
from src.mlproject1 import logger

class ConfigurationManager:
    def __init__(self,CONFIG_FILE_PATH,SCHEMA_FILE_PATH,PARAMS_FILE_PATH):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAMS_FILE_PATH)
        self.schema=read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_model_trainer(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_conf=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_csv=config.train_csv,
            test_csv=config.test_csv,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_conf

In [8]:
from sklearn.linear_model import ElasticNet
import pandas as pd
from pathlib import Path
import joblib
import os

class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config
        
    def train(self):
        schema=self.schema.TARGET_COLUMN

        train_data=pd.read_csv("artifacts/data_transformation/train.csv")
        test_data=pd.read_csv("artifacts/data_transformation/test.csv")

        X_train=train_data.drop([schema.name],axis=1)
        y_train=train_data([schema.name])

        X_test=test_data.drop([schema.name],axis=1)
        y_test=test_data([schema.name])

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        joblib.dump(lr,os.path.join([self.config.model_trainer.root_dir],[self.config.model_trainer.model_name]))




In [ ]:
try:
    con=ConfigurationManager()
    get_model_t=con.get_model_trainer()
    model_tr1=ModelTrainer(config=get_model_t)
    model_tr1.train()
except Exception as e:
    raise e